In [ ]:
from astroCAST.analysis import Video, Events, Correlation, Plotting
from pathlib import Path

import pandas as pd
import numpy as np

In [ ]:
folder = Path("/media/janrei1/data/method_testing/")

event_folder = folder.joinpath("22A7x9-1.roi") # your input file name here. 'h5' or 'tdb' is recommended for parallel processing 
assert event_folder.is_dir()

In [ ]:
events = Events(event_folder)
events.events.columns

# Exploration

In [ ]:
v = events.show_event_map(video=folder.joinpath("22A7x9-1.tiff"), z_slice=(0, 10))

In [ ]:
## summary statistics
events.get_summary_statistics().head()

In [ ]:
p = Plotting(events)

# sample traces
_ = p.plot_traces(num_samples=4)

## clustered traces
#todo clustered traces


## distribution plots
_ = p.plot_distribution("area")

## distibution without outliers
_ = p.plot_distribution("area", outlier_deviation=3)

In [ ]:
## filtering

# Correlation

# Comparison